In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [7]:
import json
lion = json.load(open('lion.geojson'))

In [62]:
lion_df = pd.DataFrame(f['properties'] for f in lion['features'])

In [104]:
lion_l = lion_df.filter(['FromLeft', 'FromRight', 'ToLeft', 'ToRight', 'SegmentID', 'StreetCode'], axis=1).copy()
lion_r = lion_l.copy()
lion_l['From'] = lion_l['FromLeft']
lion_l['To'] = lion_l['ToLeft']
lion_l['Side'] = 'L'
lion_r['From'] = lion_r['FromRight']
lion_r['To'] = lion_r['ToRight']
lion_r['Side'] = 'R'

lion_split = lion_l.append(lion_r)
lion_split['Even'] = (lion_split['From'] % 2 == 0)
lion_split = lion_split.filter(['StreetCode', 'SegmentID', 'From', 'To', 'Side', 'Even'])

In [105]:
lion_split = lion_split.sort_values(['From', 'StreetCode', 'Even'])

In [10]:
parking_violations = pd.read_csv('parking-violations.csv', nrows=100000, low_memory=False)

In [128]:
CONVERT_BOROUGH = {
    'K': 3,
    'Q': 4,
    'NY': 1,
    'BX': 2,
    'R': 5,
}
def get_full_street_code(row):
    stc1 = int(row['Street Code1'])
    if stc1 == 0:
        return -1
    if row['Violation County'] not in CONVERT_BOROUGH:
        return -1
    return str(stc1 + CONVERT_BOROUGH[row['Violation County']] * 100000)

def addr(row):
    addr_str = str(row['House Number'])
    try:
        if addr_str.find('-') >= 0:
            above, below = addr_str.split('-')
            addr = int(above) * 1000 + int(below)
        elif addr_str == 'nan':
            return -1
        else:
            addr = int(addr_str)
    except ValueError:
        # TODO: try to clean up non-numeric addresses
        return -1
    return addr

parking_violations['StreetCode'] = parking_violations.apply(get_full_street_code, axis=1)
parking_violations['Addr'] = parking_violations.apply(addr, axis=1)
parking_violations['Even'] = (parking_violations['Addr'] % 2 == 0)

In [129]:
parking_violations_cleaned = parking_violations[
    (parking_violations['Addr'] > -1) &
    parking_violations['StreetCode'].notna()].copy().sort_values(
    ['Addr', 'StreetCode', 'Even'])

In [130]:
pv_merged = pd.merge_asof(
    parking_violations_cleaned,
    lion_split, 
    by=['StreetCode', 'Even'], 
    right_on='From', left_on='Addr')
pv_merged = pv_merged[pv_merged['Addr'] <= pv_merged['To']]
pv_merged

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Hydrant Violation,Double Parking Violation,FullStreetCode,Addr,Even,StreetCode,SegmentID,From,To,Side
30,1434837993,HES4672,NY,PAS,06/12/2018,38,SDN,FORD,X,85,...,NaN,NaN,None,1,False,100085,0036620,1.0,3.0,R
31,1434838092,HTD4434,NY,PAS,06/14/2018,38,SUBN,HONDA,X,85,...,NaN,NaN,None,1,False,100085,0036620,1.0,3.0,R
32,1434838109,HLP1585,NY,PAS,06/14/2018,38,SUBN,AUDI,X,85,...,NaN,NaN,None,1,False,100085,0036620,1.0,3.0,R
33,1434838780,HLP1585,NY,PAS,06/19/2018,38,NaN,AUDI,X,85,...,NaN,NaN,None,1,False,100085,0036620,1.0,3.0,R
34,1434839801,HLP1535,NY,PAS,06/25/2018,38,SUBN,BMW,X,85,...,NaN,NaN,None,1,False,100085,0036620,1.0,3.0,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39209,1440160960,HLC8082,NY,PAS,07/12/2018,40,SUBN,HONDA,P,40420,...,NaN,NaN,440420,256020,True,440420,0062634,256000.0,256028.0,R
39210,1440158514,GBJ3186,NY,PAS,06/24/2018,27,VAN,HONDA,P,65590,...,NaN,NaN,465590,258001,False,465590,0146252,258001.0,258017.0,L
39211,1440160510,HSB2599,NY,PAS,07/03/2018,40,SUBN,FORD,P,23050,...,NaN,NaN,423050,258031,False,423050,0062668,258001.0,258031.0,L
39212,1440160399,GCY6049,NY,PAS,07/11/2018,98,SDN,TOYOT,P,23060,...,NaN,NaN,423060,259015,False,423060,0062699,259001.0,259073.0,L


In [131]:
import shapely.geometry
lion_geom = {
    f['properties']['SegmentID'] : shapely.geometry.MultiLineString(f['geometry']['coordinates'])
    for f in lion['features']
}

In [33]:
from pyproj import CRS, Transformer
wgs84 = CRS.from_epsg(4326)
nySP = CRS.from_epsg(2263)
transformer = Transformer.from_crs(nySP, wgs84)

In [ ]:
def find_latlng(row):
    if row['To'] == row['From']:
        fraction = 0
    else:
        fraction = (row['Addr'] - row['From']) / (row['To'] - row['From'])
    coords = lion_geom[row['SegmentID']].interpolate(fraction, True).coords[0]
    geom = transformer.transform(coords[0], coords[1])
    return '%0.6f,%0.6f' % geom

pv_merged['LatLng'] = pv_merged.apply(find_latlng, axis=1)
pv_merged[['Lat', 'Lng']] = pv_merged['LatLng'].str.split(',', n=1, expand=True).drop(['LatLng'])
# list(lion_df.columns)
# will join dataset of parking violations columns Registration State, 

In [127]:
pv_merged.to_csv('latlng.csv')